<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import copy
import math
import numpy as np
import time

# --- I. THÔNG SỐ CỐ ĐỊNH (TĨNH) ---
KY_HIEU_X = "X" # Người chơi MAX (AI cố gắng đạt điểm +1)
KY_HIEU_O = "O" # Người chơi MIN (AI cố gắng đạt điểm -1)
O_TRONG = None
# KICH_THUOC sẽ được xác định bằng input người dùng

# --- II. CÁC HÀM CƠ BẢN (TRÒ CHƠI N x N) ---

def trang_thai_ban_dau(N):
    """Tác dụng Code: Khởi tạo bàn cờ N x N rỗng."""
    return [[O_TRONG for _ in range(N)] for _ in range(N)]

def nguoi_choi(ban_co):
    """Tác dụng Code: Xác định lượt chơi hiện tại."""
    dem_x = sum(hang.count(KY_HIEU_X) for hang in ban_co)
    dem_o = sum(hang.count(KY_HIEU_O) for hang in ban_co)
    return KY_HIEU_X if dem_x == dem_o else KY_HIEU_O

def cac_hanh_dong(ban_co):
    """Tác dụng Code: Trả về các nước đi hợp lệ (vị trí còn trống)."""
    N = len(ban_co) # Lấy kích thước N từ ma trận
    # Dữ liệu Động 4: Tập hợp các hành động khả thi
    return set((i, j) for i in range(N) for j in range(N) if ban_co[i][j] == O_TRONG)

def ket_qua_nuoc_di(ban_co, hanh_dong):
    """Tác dụng Code: Tạo ra trạng thái bàn cờ MỚI sau khi thực hiện hành động."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    # Note: Dùng copy.deepcopy() để đảm bảo trạng thái MỚI độc lập (không ảnh hưởng đến trạng thái cũ).
    ban_co_moi = copy.deepcopy(ban_co)
    (i, j) = hanh_dong
    # Dữ liệu Động 1: Cập nhật ma trận bàn cờ
    ban_co_moi[i][j] = nguoi_choi_hien_tai
    return ban_co_moi

def diem_so(ban_co):
    """Tác dụng Code: Trả về điểm (giá trị) cuối cùng của trò chơi."""
    nguoi_thang = tim_nguoi_thang(ban_co)
    if nguoi_thang == KY_HIEU_X: return 1
    elif nguoi_thang == KY_HIEU_O: return -1
    return 0

# (Các hàm tim_nguoi_thang và ket_thuc được giữ nguyên logic)
def tim_nguoi_thang(ban_co):
    """Tác dụng Code: Kiểm tra chiến thắng (N x N)."""
    N = len(ban_co)

    # Kiểm tra Hàng và Cột (logic đã được sửa để hoạt động với N bất kỳ)
    for i in range(N):
        hang_i = ban_co[i]
        cot_i = [ban_co[r][i] for r in range(N)]

        if hang_i.count(KY_HIEU_X) == N and KY_HIEU_X in hang_i: return KY_HIEU_X
        if hang_i.count(KY_HIEU_O) == N and KY_HIEU_O in hang_i: return KY_HIEU_O

        if cot_i.count(KY_HIEU_X) == N and KY_HIEU_X in cot_i: return KY_HIEU_X
        if cot_i.count(KY_HIEU_O) == N and KY_HIEU_O in cot_i: return KY_HIEU_O

    # Kiểm tra Chéo chính và Chéo phụ
    cheo_chinh = [ban_co[i][i] for i in range(N)]
    if cheo_chinh.count(KY_HIEU_X) == N and KY_HIEU_X in cheo_chinh: return KY_HIEU_X
    if cheo_chinh.count(KY_HIEU_O) == N and KY_HIEU_O in cheo_chinh: return KY_HIEU_O

    cheo_phu = [ban_co[i][N - 1 - i] for i in range(N)]
    if cheo_phu.count(KY_HIEU_X) == N and KY_HIEU_X in cheo_phu: return KY_HIEU_X
    if cheo_phu.count(KY_HIEU_O) == N and KY_HIEU_O in cheo_phu: return KY_HIEU_O

    return None

def ket_thuc(ban_co):
    if tim_nguoi_thang(ban_co) is not None: return True
    return not any(O_TRONG in hang for hang in ban_co)

# --- III. THUẬT TOÁN MINIMAX VỚI ALPHA-BETA PRUNING (Tối ưu hóa) ---

def gia_tri_toi_da(trang_thai, alpha, beta):
    """Hàm Maximize (Người chơi X) - Cố gắng tối đa hóa điểm số."""
    if ket_thuc(trang_thai): return diem_so(trang_thai)

    v = -math.inf
    # Dữ liệu Động 2: Giá trị Alpha (α) - Giá trị TỐT NHẤT mà MAX đã tìm được cho đến nay.
    for hanh_dong in cac_hanh_dong(trang_thai):
        v = max(v, gia_tri_toi_thieu(ket_qua_nuoc_di(trang_thai, hanh_dong), alpha, beta))

        alpha = max(alpha, v)
        # Tác dụng Code (Cắt tỉa Alpha): Nếu alpha >= beta, dừng tìm kiếm nhánh này (Pruning).
        if alpha >= beta:
            break

    return v

def gia_tri_toi_thieu(trang_thai, alpha, beta):
    """Hàm Minimize (Người chơi O) - Cố gắng tối thiểu hóa điểm số."""
    if ket_thuc(trang_thai): return diem_so(trang_thai)

    v = math.inf
    # Dữ liệu Động 3: Giá trị Beta (β) - Giá trị TỐT NHẤT mà MIN đã tìm được cho đến nay.
    for hanh_dong in cac_hanh_dong(trang_thai):
        v = min(v, gia_tri_toi_da(ket_qua_nuoc_di(trang_thai, hanh_dong), alpha, beta))

        beta = min(beta, v)
        # Tác dụng Code (Cắt tỉa Beta): Nếu alpha >= beta, dừng tìm kiếm nhánh này (Pruning).
        if alpha >= beta:
            break

    return v

def minimax(ban_co):
    """Tác dụng Code: Trả về nước đi tối ưu (hành động) bằng cách gọi Minimax đệ quy."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    alpha = -math.inf
    beta = math.inf

    # Note: Vòng lặp ngoài cùng này tìm hành động tối ưu dựa trên điểm số (gia_tri) trả về từ Alpha-Beta.

    if nguoi_choi_hien_tai == KY_HIEU_X:
        diem_tot_nhat = -math.inf
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_thieu(ket_qua_nuoc_di(ban_co, hanh_dong), alpha, beta)
            if gia_tri > diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
            alpha = max(alpha, diem_tot_nhat)
        return hanh_dong_tot_nhat

    else: # người chơi hiện tại là O
        diem_tot_nhat = math.inf
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_da(ket_qua_nuoc_di(ban_co, hanh_dong), alpha, beta)
            if gia_tri < diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
            beta = min(beta, diem_tot_nhat)
        return hanh_dong_tot_nhat

# --- IV. KHỐI THỰC THI TRÒ CHƠI (MAIN) ---

def thuc_thi_chinh():

    # Dữ liệu Động 5: Kích thước N (Dynamic Input)
    while True:
        try:
            N = int(input("Nhập kích thước bàn cờ N x N (ví dụ: 3): "))
            if N >= 3:
                break
            else:
                print("Kích thước phải lớn hơn hoặc bằng 3.")
        except ValueError:
            print("Vui lòng nhập một số nguyên.")

    # Cảnh báo về hiệu suất
    if N > 3:
        print(f"\nCẢNH BÁO: Với kích thước {N}x{N}, thuật toán Minimax sẽ RẤT CHẬM (có thể treo)!")
        time.sleep(1)

    # Dữ liệu Động 6: Khởi tạo trạng thái ban đầu
    ban_co = trang_thai_ban_dau(N)

    # Dữ liệu Động 7: Vai trò của Người dùng và AI (Input từ người dùng)
    while True:
        lua_chon = input("Bạn muốn chơi với vai trò X hay O? (X/O): ").upper()
        if lua_chon in [KY_HIEU_X, KY_HIEU_O]:
            nguoi_dung = lua_chon
            ai = KY_HIEU_O if nguoi_dung == KY_HIEU_X else KY_HIEU_X
            break
        print("Lựa chọn không hợp lệ. Vui lòng nhập X hoặc O.")

    print("="*50)
    print(f"--- TRÒ CHƠI TIC TAC TOE {N}x{N} ---")
    print(f"Bạn: {nguoi_dung}, Máy tính (AI): {ai}")
    print("="*50)

    # Vòng lặp trò chơi
    while True:
        luot_hien_tai = nguoi_choi(ban_co)

        print(f"\n--- Bàn cờ hiện tại (Kích thước {N}x{N}) ---")
        # Tác dụng Code: Hiển thị bàn cờ rõ ràng cho việc demo.
        print(np.array(ban_co))

        if ket_thuc(ban_co):
            nguoi_thang = tim_nguoi_thang(ban_co)
            if nguoi_thang is None: print("Trò chơi kết thúc: HÒA.")
            else: print(f"Trò chơi kết thúc: {nguoi_thang} THẮNG.")
            break

        # Lượt của AI
        if luot_hien_tai == ai:
            print(f"Máy tính ({ai}) đang tính toán nước đi tối ưu (Alpha-Beta)...")
            start_time = time.time()
            # Dữ liệu Động 8: Nước đi AI (Kết quả của thuật toán)
            nuoc_di_ai = minimax(ban_co)
            end_time = time.time()

            # Dữ liệu Động 1: Cập nhật ma trận bàn cờ
            ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_ai)
            print(f"Máy tính đi: Hàng {nuoc_di_ai[0]}, Cột {nuoc_di_ai[1]}")
            print(f"Thời gian tính toán: {end_time - start_time:.4f} giây.")

        # Lượt của Người chơi
        else:
            print(f"Lượt của bạn ({nguoi_dung})")
            while True:
                try:
                    # Dữ liệu Động 9: Input Tọa độ (i, j)
                    i = int(input(f"Nhập Hàng (0-{N-1}): "))
                    j = int(input(f"Nhập Cột (0-{N-1}): "))
                    nuoc_di_nguoi_dung = (i, j)

                    if 0 <= i < N and 0 <= j < N and nuoc_di_nguoi_dung in cac_hanh_dong(ban_co):
                        ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_nguoi_dung)
                        break
                    else:
                        print("Nước đi không hợp lệ (ngoài phạm vi hoặc ô đã bị chiếm). Vui lòng thử lại.")
                except ValueError:
                    print("Vui lòng chỉ nhập số nguyên.")

thuc_thi_chinh()

Nhập kích thước bàn cờ N x N (ví dụ: 3): 3
Bạn muốn chơi với vai trò X hay O? (X/O): x
--- TRÒ CHƠI TIC TAC TOE 3x3 ---
Bạn: X, Máy tính (AI): O

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[[None None None]
 [None None None]
 [None None None]]
Lượt của bạn (X)
Nhập Hàng (0-2): 1
Nhập Cột (0-2): 1

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[[None None None]
 [None 'X' None]
 [None None None]]
Máy tính (O) đang tính toán nước đi tối ưu (Alpha-Beta)...
Máy tính đi: Hàng 0, Cột 0
Thời gian tính toán: 0.0328 giây.

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[['O' None None]
 [None 'X' None]
 [None None None]]
Lượt của bạn (X)
Nhập Hàng (0-2): 0
Nhập Cột (0-2): 1

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[['O' 'X' None]
 [None 'X' None]
 [None None None]]
Máy tính (O) đang tính toán nước đi tối ưu (Alpha-Beta)...
Máy tính đi: Hàng 2, Cột 1
Thời gian tính toán: 0.0024 giây.

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[['O' 'X' None]
 [None 'X' None]
 [None 'O' None]]
Lượt của bạn (X)
Nhập Hàng (0-2)

In [7]:
import copy
import math
import numpy as np
import time

# --- I. THÔNG SỐ CỐ ĐỊNH (TĨNH) ---
KY_HIEU_X = "X" # Người chơi MAX (AI cố gắng đạt điểm +1)
KY_HIEU_O = "O" # Người chơi MIN (AI cố gắng đạt điểm -1)
O_TRONG = None  # Đại diện cho ô trống trên bàn cờ
# Kích thước N x N sẽ được xác định bằng input người dùng (Dynamic Input)

# --- II. CÁC HÀM CƠ BẢN (TRÒ CHƠI N x N) ---

def trang_thai_ban_dau(N):
    """Tác dụng Code: Khởi tạo ma trận bàn cờ N x N rỗng."""
    return [[O_TRONG for _ in range(N)] for _ in range(N)]

def nguoi_choi(ban_co):
    """Tác dụng Code: Xác định lượt chơi tiếp theo (X luôn đi trước)."""
    dem_x = sum(h.count(KY_HIEU_X) for h in ban_co)
    dem_o = sum(h.count(KY_HIEU_O) for h in ban_co)
    return KY_HIEU_X if dem_x == dem_o else KY_HIEU_O

def cac_hanh_dong(ban_co):
    """Tác dụng Code: Trả về tập hợp tất cả các nước đi hợp lệ (tọa độ ô trống)."""
    N = len(ban_co)
    # Dữ liệu Động 4: Tập hợp các hành động khả thi. Thay đổi tùy thuộc vào trạng thái bàn cờ.
    return set((i, j) for i in range(N) for j in range(N) if ban_co[i][j] == O_TRONG)

def ket_qua_nuoc_di(ban_co, hanh_dong):
    """Tác dụng Code: Tạo ra trạng thái bàn cờ MỚI sau khi người chơi hiện tại thực hiện 'hanh_dong'."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    ban_co_moi = copy.deepcopy(ban_co)
    (i, j) = hanh_dong
    # Dữ liệu Động 1: Cập nhật ma trận bàn cờ tại tọa độ (i, j)
    ban_co_moi[i][j] = nguoi_choi_hien_tai
    return ban_co_moi

def tim_nguoi_thang(ban_co):
    """Tác dụng Code: Kiểm tra điều kiện thắng (N quân liên tiếp) trên bàn cờ N x N."""
    N = len(ban_co)

    # Duyệt qua N hàng và N cột
    for i in range(N):
        # 1. Kiểm tra Hàng (Ngang)
        if all(ban_co[i][j] == ban_co[i][0] for j in range(N)) and ban_co[i][0] is not O_TRONG: return ban_co[i][0]
        # 2. Kiểm tra Cột (Dọc)
        if all(ban_co[r][i] == ban_co[0][i] for r in range(N)) and ban_co[0][i] is not O_TRONG: return ban_co[0][i]

    # 3. Kiểm tra Chéo chính (\)
    cheo_chinh = [ban_co[i][i] for i in range(N)]
    if all(c == cheo_chinh[0] for c in cheo_chinh) and cheo_chinh[0] is not O_TRONG: return cheo_chinh[0]

    # 4. Kiểm tra Chéo phụ (/)
    cheo_phu = [ban_co[i][N - 1 - i] for i in range(N)]
    if all(c == cheo_phu[0] for c in cheo_phu) and cheo_phu[0] is not O_TRONG: return cheo_phu[0]

    return None

def ket_thuc(ban_co):
    """Tác dụng Code: Kiểm tra trò chơi đã kết thúc (có người thắng hoặc hết ô trống)."""
    if tim_nguoi_thang(ban_co) is not None: return True
    # Trả về True nếu không còn ô trống nào (HÒA)
    return not any(O_TRONG in hang for hang in ban_co)

def diem_so(ban_co):
    """Tác dụng Code: Trả về điểm (giá trị) cuối cùng: +1 (X thắng), -1 (O thắng), 0 (Hòa)."""
    nguoi_thang = tim_nguoi_thang(ban_co)
    if nguoi_thang == KY_HIEU_X: return 1
    elif nguoi_thang == KY_HIEU_O: return -1
    return 0

# --- III. THUẬT TOÁN MINIMAX VỚI ALPHA-BETA PRUNING (Tối ưu hóa) ---

def gia_tri_toi_da(trang_thai, alpha, beta):
    """
    Hàm Maximize (Người chơi X) - Cố gắng tối đa hóa điểm số.
    Thuật toán: Lựa chọn Max(điểm số của tất cả các nước đi khả thi).
    """
    if ket_thuc(trang_thai): return diem_so(trang_thai)
    v = -math.inf
    # Dữ liệu Động 2: Alpha (α) - Giá trị tốt nhất MAX đã thấy trên đường đi hiện tại.
    for hanh_dong in cac_hanh_dong(trang_thai):
        # Đệ quy: Gọi hàm gia_tri_toi_thieu (lượt của đối thủ O)
        v = max(v, gia_tri_toi_thieu(ket_qua_nuoc_di(trang_thai, hanh_dong), alpha, beta))

        alpha = max(alpha, v) # Cập nhật Alpha
        # Tác dụng Code (Cắt tỉa Alpha): Nếu alpha >= beta, nhánh này bị cắt (pruning).
        # Tức là: Nếu điểm số tối thiểu (β) mà MIN có thể đạt được đã nhỏ hơn
        # điểm số tối đa (α) mà MAX đã đảm bảo, thì MIN sẽ không bao giờ để MAX đạt đến trạng thái này.
        if alpha >= beta: break

    return v

def gia_tri_toi_thieu(trang_thai, alpha, beta):
    """
    Hàm Minimize (Người chơi O) - Cố gắng tối thiểu hóa điểm số.
    Thuật toán: Lựa chọn Min(điểm số của tất cả các nước đi khả thi).
    """
    if ket_thuc(trang_thai): return diem_so(trang_thai)
    v = math.inf
    # Dữ liệu Động 3: Beta (β) - Giá trị tốt nhất MIN đã thấy trên đường đi hiện tại.
    for hanh_dong in cac_hanh_dong(trang_thai):
        # Đệ quy: Gọi hàm gia_tri_toi_da (lượt của đối thủ X)
        v = min(v, gia_tri_toi_da(ket_qua_nuoc_di(trang_thai, hanh_dong), alpha, beta))

        beta = min(beta, v) # Cập nhật Beta
        # Tác dụng Code (Cắt tỉa Beta): Nếu alpha >= beta, nhánh này bị cắt (pruning).
        # Tức là: Nếu điểm số tối đa (α) mà MAX có thể đạt được đã lớn hơn
        # điểm số tối thiểu (β) mà MIN đã đảm bảo, thì MAX sẽ không bao giờ đi vào nhánh này.
        if alpha >= beta: break

    return v

def minimax(ban_co):
    """Tác dụng Code: Hàm gốc, tìm nước đi TỐI ƯU (optimal action) bằng cách gọi đệ quy Alpha-Beta."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    alpha = -math.inf # Khởi tạo Alpha ban đầu là âm vô cực
    beta = math.inf   # Khởi tạo Beta ban đầu là dương vô cực
    hanh_dong_tot_nhat = None

    if nguoi_choi_hien_tai == KY_HIEU_X:
        diem_tot_nhat = -math.inf
        for hanh_dong in cac_hanh_dong(ban_co):
            # Lượt của X, gọi gia_tri_toi_thieu để đối thủ O đánh giá nước đi
            gia_tri = gia_tri_toi_thieu(ket_qua_nuoc_di(ban_co, hanh_dong), alpha, beta)
            if gia_tri > diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
            # Cập nhật alpha ở tầng gốc để giới hạn tìm kiếm cho các nhánh tiếp theo.
            alpha = max(alpha, diem_tot_nhat)
        return hanh_dong_tot_nhat

    else: # người chơi hiện tại là O
        diem_tot_nhat = math.inf
        for hanh_dong in cac_hanh_dong(ban_co):
            # Lượt của O, gọi gia_tri_toi_da để đối thủ X đánh giá nước đi
            gia_tri = gia_tri_toi_da(ket_qua_nuoc_di(ban_co, hanh_dong), alpha, beta)
            if gia_tri < diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
            # Cập nhật beta ở tầng gốc để giới hạn tìm kiếm cho các nhánh tiếp theo.
            beta = min(beta, diem_tot_nhat)
        return hanh_dong_tot_nhat

# --- IV. HÀM NHẬP DỮ LIỆU ĐỘNG VÀ KHỐI THỰC THI (MAIN) ---

def nhap_trang_thai_dau(N):
    """Tác dụng Code: Lấy ma trận trạng thái ban đầu N x N từ người dùng."""
    print(f"\n--- NHẬP TRẠNG THÁI BÀN CỜ {N}x{N} BAN ĐẦU (DYNAMIC INPUT) ---")
    print("Quy ước: X (Người chơi X), O (Người chơi O), E (Ô Trống - Empty).")

    ban_co = []
    for i in range(N):
        while True:
            try:
                # Nhập chuỗi
                hang_nhap = input(f"Nhập Hàng {i} (Ví dụ: X O E X): ").upper().split()
                # Kiểm tra số lượng ký hiệu
                if len(hang_nhap) != N: raise ValueError(f"Cần đúng {N} ký hiệu.")

                hang_ma_hoa = []
                for ky_hieu in hang_nhap:
                    if ky_hieu == KY_HIEU_X: hang_ma_hoa.append(KY_HIEU_X)
                    elif ky_hieu == KY_HIEU_O: hang_ma_hoa.append(KY_HIEU_O)
                    elif ky_hieu == 'E': hang_ma_hoa.append(O_TRONG)
                    else: raise ValueError(f"Ký hiệu không hợp lệ: {ky_hieu}")

                ban_co.append(hang_ma_hoa)
                break
            except ValueError as e:
                print(f"Lỗi: {e}. Vui lòng thử lại.")
            except:
                print("Lỗi nhập liệu không xác định.")

    return ban_co

def thuc_thi_chinh():

    # Dữ liệu Động 5: Kích thước N. Lấy input N từ người dùng.
    while True:
        try:
            N = int(input("Nhập kích thước bàn cờ N x N (ví dụ: 3): "))
            if N >= 3: break
            else: print("Kích thước phải lớn hơn hoặc bằng 3.")
        except ValueError: print("Vui lòng nhập một số nguyên.")

    # Cảnh báo về hiệu suất đối với N lớn
    if N > 3: print(f"\nCẢNH BÁO: Với kích thước {N}x{N}, thuật toán Minimax sẽ RẤT CHẬM (có thể treo)!")

    # Lựa chọn trạng thái ban đầu (Dynamic Start State)
    chon_trang_thai = input("Bạn muốn (1) Bàn cờ RỖNG hay (2) Nhập TRẠNG THÁI BAN ĐẦU? (1/2): ")

    if chon_trang_thai == '2':
        # Dữ liệu Động 6: Ma trận trạng thái ban đầu được nhập từ người dùng.
        ban_co = nhap_trang_thai_dau(N)
    else:
        # Dữ liệu Động 6: Ma trận trạng thái ban đầu rỗng.
        ban_co = trang_thai_ban_dau(N)

    # Dữ liệu Động 7: Vai trò của Người dùng và AI. Lấy input X/O từ người dùng.
    while True:
        lua_chon = input("Bạn muốn chơi với vai trò X hay O? (X/O): ").upper()
        if lua_chon in [KY_HIEU_X, KY_HIEU_O]:
            nguoi_dung = lua_chon
            ai = KY_HIEU_O if nguoi_dung == KY_HIEU_X else KY_HIEU_X
            break
        print("Lựa chọn không hợp lệ. Vui lòng nhập X hoặc O.")

    print("="*50)
    print(f"--- TRÒ CHƠI TIC TAC TOE {N}x{N} ---")
    print(f"Bạn: {nguoi_dung}, Máy tính (AI): {ai}")
    print("="*50)

    # Vòng lặp trò chơi chính
    while True:
        luot_hien_tai = nguoi_choi(ban_co)

        print(f"\n--- Bàn cờ hiện tại (Kích thước {N}x{N}) ---")
        # In bàn cờ ra màn hình
        print(np.array(ban_co, dtype=object))

        if ket_thuc(ban_co):
            nguoi_thang = tim_nguoi_thang(ban_co)
            if nguoi_thang is None: print("Trò chơi kết thúc: HÒA.")
            else: print(f"Trò chơi kết thúc: {nguoi_thang} THẮNG.")
            break

        # Lượt của AI (Sử dụng thuật toán)
        if luot_hien_tai == ai:
            print(f"Máy tính ({ai}) đang tính toán nước đi tối ưu (Alpha-Beta)...")
            start_time = time.time()
            # Dữ liệu Động 8: Nước đi AI (Kết quả của thuật toán Minimax).
            nuoc_di_ai = minimax(ban_co)
            end_time = time.time()

            # Dữ liệu Động 1: Cập nhật ma trận bàn cờ sau nước đi của AI
            ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_ai)
            print(f"Máy tính đi: Hàng {nuoc_di_ai[0]}, Cột {nuoc_di_ai[1]}")
            print(f"Thời gian tính toán: {end_time - start_time:.4f} giây.")

        # Lượt của Người chơi (Lấy input tọa độ)
        else:
            print(f"Lượt của bạn ({nguoi_dung})")
            while True:
                try:
                    # Dữ liệu Động 9: Input Tọa độ (i, j) của người dùng.
                    i = int(input(f"Nhập Hàng (0-{N-1}): "))
                    j = int(input(f"Nhập Cột (0-{N-1}): "))
                    nuoc_di_nguoi_dung = (i, j)

                    # Kiểm tra nước đi hợp lệ (trong phạm vi và ô trống)
                    if 0 <= i < N and 0 <= j < N and nuoc_di_nguoi_dung in cac_hanh_dong(ban_co):
                        ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_nguoi_dung)
                        break
                    else:
                        print("Nước đi không hợp lệ (ngoài phạm vi hoặc ô đã bị chiếm). Vui lòng thử lại.")
                except ValueError:
                    print("Lỗi: Vui lòng chỉ nhập số nguyên.")

thuc_thi_chinh()

Nhập kích thước bàn cờ N x N (ví dụ: 3): 3
Bạn muốn (1) Bàn cờ RỖNG hay (2) Nhập TRẠNG THÁI BAN ĐẦU? (1/2): 2

--- NHẬP TRẠNG THÁI BÀN CỜ 3x3 BAN ĐẦU (DYNAMIC INPUT) ---
Quy ước: X (Người chơi X), O (Người chơi O), E (Ô Trống - Empty).
Nhập Hàng 0 (Ví dụ: X O E X): x o e
Nhập Hàng 1 (Ví dụ: X O E X): o x e 
Nhập Hàng 2 (Ví dụ: X O E X): e o x
Bạn muốn chơi với vai trò X hay O? (X/O): x
--- TRÒ CHƠI TIC TAC TOE 3x3 ---
Bạn: X, Máy tính (AI): O

--- Bàn cờ hiện tại (Kích thước 3x3) ---
[['X' 'O' None]
 ['O' 'X' None]
 [None 'O' 'X']]
Trò chơi kết thúc: X THẮNG.
